# BLAR - SOBI-FromAll model

- Notebook thực hiện các bước để finetune model nhận diện chữ viết tắt/định nghĩa

- Notebook này tham khảo phần code để huấn luyện mô hình BERT của notebook: https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/BERT/Custom_Named_Entity_Recognition_with_BERT_only_first_wordpiece.ipynb




# Chuẩn bị các thư viện cần thiết

Cài đặt thư viện bioc và transformers

In [2]:
pip install bioc transformers

     |████████████████████████████████| 4.0 MB 7.0 MB/s 
     |████████████████████████████████| 6.4 MB 54.9 MB/s 
     |████████████████████████████████| 895 kB 69.6 MB/s 
     |████████████████████████████████| 77 kB 7.3 MB/s 
     |████████████████████████████████| 6.6 MB 54.0 MB/s 
     |████████████████████████████████| 596 kB 72.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [3]:
# import các thư viện
import os
import transformers
import bioc
import math
import torch
import pandas as pd
import numpy as np
from copy import deepcopy
from torch.utils.data import Dataset, DataLoader
from transformers import BertForTokenClassification, AutoModelForTokenClassification
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt

In [4]:
# Kiểm tra colab sử dụng GPU để tăng tốc độ train
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


# Import data

In [16]:
#@title Thực hiện import data bằng 1 trong 2 cách
#@markdown Cách 1: tải folder BLAR về google drive

#@markdown Cách 2: huấn luyện thử mô hình, tải dữ liệu về ổ cứng tạm thời của colab

method = "download_to_temporary_disk" #@param ["download_to_google_drive", "download_to_temporary_disk"]


if method == 'download_to_google_drive':
  
  from google.colab import drive
  drive.mount('/content/drive')
  os.chdir('/content/drive/MyDrive/Colab Notebooks/BLAR data') ##Đổi về directory trong drive

else:
  os.chdir('/content')
  directory = './corpus'
  if not os.path.exists(directory):
      os.makedirs(directory)
  os.chdir(directory)
  ## Download data về disk
  !gdown --id 16hOCtaVyuE2LK_n7mHNO_ptm_A-A8kjV
  !gdown --id 1Mj80pavNCcIHB7_e59DVQyv0EoS8bI6a
  os.chdir('/content')

  directory = './model/model_SOBI'
  if not os.path.exists(directory):
      os.makedirs(directory)
  os.chdir(directory)
  ## Download data về disk
  !gdown --id 1Xg7XBdIWs5k76KEB9J1pG8HikPsIdnKM
  !gdown --id 1rjDB8_GmTAvBrzu8qp7TfxORl0zIMm1b
  !gdown --id 1MvJ0Auu0l_XrjcM9prj0xj-E9ckMXO98
  os.chdir('/content')

  directory = './model/2stepFromAllSentenceModel'
  if not os.path.exists(directory):
      os.makedirs(directory)
  os.chdir(directory)
  ## Download data về disk
  !gdown --id 1Lut7rci259_d3waHkdRjS3MGLwBN02Pe
  !gdown --id 1AIBtnfEYgJlmXFBzT0kZrEVnnBfyTjTL
  !gdown --id 10-U7cjGOiZ6Nty7OC3BCtyswP6ilk4VY
  os.chdir('/content')

## Load data từ google drive
### BIOADI corpus
from bioc import biocxml
with open('./corpus/bioadi_bioc_gold.xml', 'r') as fp:
  gold_raw = biocxml.load(fp)

### AB3P corpus
from bioc import biocxml
with open('./corpus/Ab3P_bioc_gold.xml', 'r') as fp:
  ab3p = biocxml.load(fp)


/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=16hOCtaVyuE2LK_n7mHNO_ptm_A-A8kjV
To: /content/corpus/bioadi_bioc_gold.xml
100% 2.58M/2.58M [00:00<00:00, 232MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Mj80pavNCcIHB7_e59DVQyv0EoS8bI6a
To: /content/corpus/Ab3P_bioc_gold.xml
100% 2.36M/2.36M [00:00<00:00, 246MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarni

In [17]:
# Tạo corpus phụ gồm các văn bản không chứa từ viết tắt
n = len(gold_raw.documents)
noAcronym = []
gold_noAcronym = bioc.bioc.BioCCollection()
for i, document in enumerate(gold_raw.documents):
  if len(document.passages[0].annotations) == 0:
    gold_noAcronym.add_document(document)
    noAcronym.append(i)

In [18]:
# Tạo corpus phụ gồm các văn bản chứa từ viết tắt
n = len(gold_raw.documents)
Acronym = []
gold = bioc.bioc.BioCCollection()
for i, document in enumerate(gold_raw.documents):
  if len(document.passages[0].annotations) != 0:
    gold.add_document(document)
    Acronym.append(i)

#Load model

In [19]:
# Load model for token classification
directory = "./model/2stepFromAllSentenceModel" # sử dụng model FromAllSentence
tokenizer = transformers.AutoTokenizer.from_pretrained(directory, local_files_only=True)
model = transformers.AutoModelForTokenClassification.from_pretrained(directory, local_files_only=True)
model.to(device)

# decoding dict for token model
labels_to_ids = {'B-LF':0,
                 'I-LF':1,
                 'L-LF':2,
                 'O':3,
                 'U-SF':4,
                 'U-PR':5,
                 'TAG':6}
ids_to_labels = {v:k for k,v in labels_to_ids.items()}

In [20]:
# Load model for sentence classification
directory = "./model/model_SOBI" # sử dụng model SOBI
tokenizer = transformers.AutoTokenizer.from_pretrained(directory, local_files_only=True)
model_seq = transformers.AutoModelForTokenClassification.from_pretrained(directory, local_files_only=True)
model_seq.to(device)

# decoding dict for sentence model
labels_to_ids_sentence = {'B':0,'I':1,'S':2, 'O':3,'SpecialToken':-100}
ids_to_labels_sentence = {0:'B',1:'I',2:'S', 3: 'O', -100:'SpecialToken'}

# Model inference

##Functions

In [21]:
#@title Create dataframe of true LF-SF (for corpus data)
def trueLFSF(corpus,idx):
  if corpus == 'ab3p':
    passage = ab3p.documents[idx].passages[1]
  elif corpus == 'bioadi':
    passage = gold_raw.documents[idx].passages[0]
  else:
    print('no true LF-SF')

  text = passage.text
  annotations = passage.annotations
  LF = []
  SF = []

  for i, annotation in enumerate(annotations):
    if i%2 == 0:
      SF.append(annotation.text.lower())
    else:
      LF.append(annotation.text.lower())
  df = pd.DataFrame({'truelongform':LF,'trueshortform':SF})
  return df


### Create dataframe of predicted LF-SF 

In [22]:
#@title Functions for step 2
#@markdown Load các hàm cho token classification model step 2
MAX_LEN = 512
alphabet = 'abcdefghijklmnopqrstuvwxyz0123456789'

def ExtractStringFromToken(listOfToken):
  # this function will decode list of token to string of text
  string = ''
  for token in listOfToken:
    if token[0] == '[' and token[-1] == ']':
      continue
    elif token[0:2] == '##':
      string = string + token[2:]
    elif token[0] not in alphabet:
      string = string + token
    elif token[0] in alphabet and string == '':
      string = string + token
    elif token[0] in alphabet and (string[-1] not in alphabet and string[-1] not in '+)'):
      string = string + token
    else:
      string = string + ' ' + token
  return string

def ExtractLFSF(wp_preds):
  # this function will infer LF-SF pairs from list of tagged tokens
  LF = []
  SF = []
  for i in range(len(wp_preds)-1):
    if wp_preds[i][1] == 'B-LF':
      for j in range(i+1,min(i+31,len(wp_preds)-1)):
        if wp_preds[j][1] == 'L-LF':
          for k in range(j+1,min(j+5,len(wp_preds))):
            if wp_preds[k][1] == 'U-SF':
              for l in range(k, min(k+12,len(wp_preds))):
                if wp_preds[l][1] != 'U-SF':
                  if wp_preds[l][0][0:2] != '##':
                    lflist = [idx[0] for idx in wp_preds[i:k-1]]
                    longform = ExtractStringFromToken(lflist)
                    if longform != '':
                      LF.append(longform)
                    sflist = [idx[0] for idx in wp_preds[k:l]]
                    shortform = ExtractStringFromToken(sflist)
                    if shortform != '':
                      SF.append(shortform)
                  break
              break  
          break
        else:
          if wp_preds[j+1][1] == 'U-SF':
            for k in range(j+1, min(j+12,len(wp_preds))):
              if wp_preds[k][1] != 'U-SF':
                if wp_preds[k][0][0:2] != '##':
                  lflist = [idx[0] for idx in wp_preds[i:j]]
                  longform = ExtractStringFromToken(lflist)
                  if longform != '':
                    LF.append(longform)
                  sflist = [idx[0] for idx in wp_preds[j+1:k]]
                  shortform = ExtractStringFromToken(sflist)
                  if shortform != '':
                    SF.append(shortform)
                break
            break

    elif wp_preds[i][1] == 'O' and wp_preds[i][0][0:2] != '##' and wp_preds[i+1][1] == 'I-LF':
      for j in range(i+1,min(i+21,len(wp_preds)-1)):
        if wp_preds[j][1] == 'L-LF':
          for k in range(j+1,min(j+5,len(wp_preds))):
            if wp_preds[k][1] == 'U-SF':
              for l in range(k, min(k+10,len(wp_preds))):
                if wp_preds[l][1] != 'U-SF':
                  if wp_preds[l][0][0:2] != '##':
                    lflist = [idx[0] for idx in wp_preds[i:k-1]]
                    longform = ExtractStringFromToken(lflist)
                    if longform != '':
                      LF.append(longform)
                    sflist = [idx[0] for idx in wp_preds[k:l]]
                    shortform = ExtractStringFromToken(sflist)
                    if shortform != '':
                      SF.append(shortform)
                  break
              break  
          break

    elif wp_preds[i][1] == 'O' and wp_preds[i][0][0:2] == '##' and wp_preds[i+1][1] == 'I-LF':
      for j in range(i+1,min(i+21,len(wp_preds)-1)):
        if wp_preds[j][1] == 'L-LF':
          for k in range(j+1,min(j+5,len(wp_preds))):
            if wp_preds[k][1] == 'U-SF':
              for l in range(k, min(k+10,len(wp_preds))):
                if wp_preds[l][1] != 'U-SF':
                  if wp_preds[l][0][0:2] != '##':
                    lflist = [idx[0] for idx in wp_preds[i+1:k-1]]
                    longform = ExtractStringFromToken(lflist)
                    if longform != '':
                      LF.append(longform)
                    sflist = [idx[0] for idx in wp_preds[k:l]]
                    shortform = ExtractStringFromToken(sflist)
                    if shortform != '':
                      SF.append(shortform)
                  break
              break  
          break

  return pd.DataFrame({'longform':LF, 'shortform':SF})  


def PredictToken(sentence):
  #This function perform token classification with input as text
  inputs = tokenizer(sentence, 
                   return_offsets_mapping=True, 
                   padding='max_length', 
                   truncation=True, 
                   max_length=MAX_LEN,
                   return_tensors="pt")

  # move to gpu
  ids = inputs["input_ids"].to(device)
  mask = inputs["attention_mask"].to(device)
  # forward pass
  outputs = model(ids, attention_mask=mask)
  logits = outputs[0]

  active_logits = logits.view(-1, 7) # shape (batch_size * seq_len, num_labels)
  flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

  tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
  token_predictions = [ids_to_labels[i] for i in flattened_predictions.cpu().numpy()]
  wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)
  return wp_preds


def predictLFSFFromSentence(sentence):
  # This function returns predicted LF-SF pairs from text
  wp_preds = PredictToken(sentence)
  return ExtractLFSF(wp_preds)

In [23]:
#@title Functions for 2 step model
#@markdown Load các hàm cho mô hình hai bước hoàn chỉnh


def ExtractSentenceWithAcronym(original_passage):
  # this function will return new passage with annotated sentences


  passage = deepcopy(original_passage)
  text = passage.text
  # extract annotation
  sf_offset_stack_unsort = dict()
  for i, annotation in enumerate(passage.annotations):
    if i%2 == 0:
      sf_offset_stack_unsort[passage.annotations[i].total_span.offset] = i #extract the SF offset only
  offsetList = [offset for offset,i in sf_offset_stack_unsort.items()]
  
  sf_offset_stack = []
  while offsetList:
    minoffset = min(offsetList)
    idx = sf_offset_stack_unsort[minoffset]
    sf_offset_stack.append([idx, passage.annotations[idx].total_span.offset])
    offsetList.remove(minoffset)

  
  # Sentence extraction
  ## Presumption: 
  ### SF always stands before LF in corpus annotation list
  ### Sentence extraction by: '. '
  ### Sentence starts with CASED character
  ### Sentence length > 20 characters

  rawSentenceList = text.split('. ') #split passage to list of sentence
  cased_alphabet = 'ABCEDEFGHIJKLMNOPQRSTUVWXYZ'
  sentenceList = []
  for i, raw_sentence in enumerate(rawSentenceList):
    if raw_sentence[-1] != '.':
      raw_sentence += '.' # add . if sentence doesn't have
    if raw_sentence[0] in cased_alphabet and len(raw_sentence) >10:
      sentenceList.append(raw_sentence)
    else:
      if sentenceList:
        sentenceList[-1] += ' ' + raw_sentence
      else:
        sentenceList.append(raw_sentence)


  # Add annotation for sentence
  pointer = 0
  for sentence in sentenceList:
    bioc_sentence = bioc.bioc.BioCSentence()
    bioc_sentence.text = sentence
    while len(sf_offset_stack):
      if sf_offset_stack[0][1] < pointer + len(sentence):
        sf = sf_offset_stack.pop(0)
        bioc_sentence.annotations.append(passage.annotations[sf[0]]) # get SF annotation
        bioc_sentence.annotations.append(passage.annotations[sf[0]+1]) # get LF annotation
        bioc_sentence.annotations[-2].locations[0].offset -= pointer # change SF offset to sentence (originally was offset passage)
        bioc_sentence.annotations[-1].locations[0].offset -= pointer # change LF offset to sentence (originally was offset passage)  
      else:
        break
    bioc_sentence.offset = pointer
    pointer += len(sentence) + 1 # move pointer to next sentence, 1 for .
    passage.add_sentence(bioc_sentence) # add sentence to passage
      
  return passage
        

def PredictTokenSentence(text):
  # This function returns predictions of token in Sentence Classification task
  text = text.split(' ')
  inputs = tokenizer(text,
                    is_split_into_words=True,
                    return_offsets_mapping=True, 
                    padding='max_length', 
                    truncation=True, 
                    max_length=512,
                    return_tensors="pt")
  # move to gpu
  ids = inputs["input_ids"].to(device)
  mask = inputs["attention_mask"].to(device)
  # forward pass
  outputs = model_seq(ids, attention_mask=mask)
  logits = outputs[0]

  active_logits = logits.view(-1, model_seq.num_labels) # shape (batch_size * seq_len, num_labels)
  flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

  tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
  token_predictions = [ids_to_labels_sentence[i] for i in flattened_predictions.cpu().numpy()]
  wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

  prediction = []
  for token_pred, mapping in zip(wp_preds, inputs["offset_mapping"].squeeze().tolist()):
    #only predictions on first word pieces are important
    if mapping[0] == 0 and mapping[1] != 0:
      prediction.append(token_pred[1])
    else:
      continue
  return prediction

def PredictSentenceWithABBR(text):
  # This function returns list of predicted correct sentence with LF/SF pairs
  text_split = text.split(' ')
  prediction = PredictTokenSentence(text)
  sentenceList = []
  for i in range(len(prediction)-1):
    if prediction[i] == 'B':
      for j in range(i+1, len(prediction)):
        if prediction[j] in 'BS':
          sentence = ' '.join(text_split[i:j])
          if j-i>4: # at least 5 words per sentence
            sentenceList.append(sentence)
          break
        elif j == len(prediction)-1:
          sentence = ' '.join(text_split[i:])
          sentenceList.append(sentence)
          break               
  return sentenceList
  
def predictLFSFFromPassage(passage):
  # This function returns predicted LF-SF pairs with input as text of passage
  l = [pd.DataFrame(columns = ['longform','shortform'])]
  for sentence in PredictSentenceWithABBR(passage):
    l.append(predictLFSFFromSentence(sentence))
  if l:
    return pd.concat(l, ignore_index = True)

### So sánh cặp từ viết tắt/định nghĩa dự đoán và từ giá trị thực

In [24]:
#@title Functions
def BuildContigencyTable(sentence,corpus,idx):
  prediction = predictLFSFFromPassage(sentence)
  truelabels = trueLFSF(corpus,idx)
  predictionList = [tuple(prediction.loc[i,]) for i in range(len(prediction))]
  truelabelsList = [tuple(truelabels.loc[i,]) for i in range(len(truelabels))]
  ls = []
  for item in predictionList:
    if item in truelabelsList:
      ls.append(pd.DataFrame({'longform':item[0],
                              'shortform':item[1],
                              'truelongform':item[0],
                              'trueshortform':item[1],
                              'TP':[1],'FP':[0],'FN':[0],'M':[0],'idx':[idx]}))
      truelabelsList.remove(item)
    else:
      if item[1] in list(truelabels['trueshortform']):
        ls.append(pd.DataFrame({'longform':item[0],
                                'shortform':item[1],
                                'truelongform':'NaN',
                                'trueshortform':'NaN',
                                'TP':[0],'FP':[0], 'FN':[0], 'M':[0],'idx':[idx]}))
      else:
        ls.append(pd.DataFrame({'longform':item[0],
                                'shortform':item[1],
                                'truelongform':'NaN',
                                'trueshortform':'NaN',
                                'TP':[0],'FP':[1], 'FN':[0],'M':[0],'idx':[idx]}))

  for item in truelabelsList:
    if item[1] in list(prediction['shortform']):
      ls.append(pd.DataFrame({'longform':'NaN',
                              'shortform':'NaN',
                              'truelongform':item[0],
                              'trueshortform':item[1],
                              'TP':[0],'FP':[0],'FN':[0],'M':[1],'idx':[idx]}))
    else:
      ls.append(pd.DataFrame({'longform':'NaN',
                              'shortform':'NaN',
                              'truelongform':item[0],
                              'trueshortform':item[1],
                              'TP':[0],'FP':[0],'FN':[1],'M':[0],'idx':[idx]}))
  if len(ls):
    contigencyTable = pd.concat(ls, ignore_index = True)
    return contigencyTable
  else:
    return pd.DataFrame()

In [25]:
#@title So sánh
corpus = "bioadi" #@param ["ab3p", "bioadi"]
index = 200 #@param {type:"integer"}

if corpus == 'ab3p':
  passage = ab3p.documents[index].passages[1].text
else:
  passage = gold_raw.documents[index].passages[0].text

print(f"Văn bản cần dự đoán: {passage}")
BuildContigencyTable(passage,corpus,index)


Văn bản cần dự đoán: Molecular cloning, expression, and chromosomal localization of a human brain-specific Na(+)-dependent inorganic phosphate cotransporter. We describe the molecular cloning of a cDNA encoding a human brain Na(+)-dependent inorganic phosphate (P(i)) cotransporter (hBNPI). The nucleotide and deduced amino acid sequences of hBNPI reveal a protein of 560 amino acids with six to eight putative transmembrane segments. hBNPI shares a high degree of homology with other Na(+)-dependent inorganic P(i) cotransporters, including those found in rat brain and human and rabbit kidney. Expression of hBNPI in COS-1 cells results in Na(+)-dependent P(i) uptake. Northern blot analysis demonstrates that hBNPI mRNA is expressed predominantly in brain and most abundantly in neuron-enriched regions such as the amygdala and hippocampus. Moderate levels of expression are also observed in glia-enriched areas such as the corpus callosum, and low levels are observed in the substantia nigra, sub

,longform,shortform,truelongform,trueshortform,TP,FP,FN,M,idx
0,brain na(+)-dependent inorganic phosphate(p(i)...,hbnpi,NaN,NaN,0,0,0,0,200
1,na(+)-dependent inorganic phosphate(p(i)) cotr...,hbnpi,NaN,NaN,0,0,0,0,200
2,NaN,NaN,inorganic phosphate,p(i),0,0,1,0,200
3,NaN,NaN,human brain na(+)-dependent inorganic phosphat...,hbnpi,0,0,0,1,200


# Kiểm tra độ chính xác trên ngữ liệu AB3P

In [26]:
#@title Functions
# Create contigency table for whole ab3p data
table = pd.DataFrame()
contigency_list = []
for idx in range(len(ab3p.documents)):
  if idx%50 == 0:
    print(idx, 'item done')
  sentence = ab3p.documents[idx].passages[1].text
  corpus = 'ab3p'
  contigency_list.append(BuildContigencyTable(sentence,corpus,idx))
final_contigency_table = pd.concat(contigency_list, ignore_index = True)


0 item done
50 item done
100 item done
150 item done
200 item done
250 item done
300 item done
350 item done
400 item done
450 item done
500 item done
550 item done
600 item done
650 item done
700 item done
750 item done
800 item done
850 item done
900 item done
950 item done
1000 item done
1050 item done
1100 item done
1150 item done
1200 item done


In [27]:
#@title Xem bảng so sánh dự đoán và kết quả thật
# show contigency table
final_contigency_table

,longform,shortform,truelongform,trueshortform,TP,FP,FN,M,idx
0,general anesthesia,ga,general anesthesia,ga,1,0,0,0,2
1,multiple sclerosis,ms,multiple sclerosis,ms,1,0,0,0,5
2,5-hydroxytryptamine,5ht,5-hydroxytryptamine,5ht,1,0,0,0,6
3,impedance cardiogram,icg,impedance cardiogram,icg,1,0,0,0,10
4,first derivative of icg,dz/dt,NaN,NaN,0,1,0,0,10
...,...,...,...,...,...,...,...,...,...
1497,sialyl lewis x,sle(x),sialyl lewis x,sle(x),1,0,0,0,1242
1498,transcriptional regulatory network,trn,transcriptional regulatory network,trn,1,0,0,0,1245
1499,methoxychlor,mxc,methoxychlor,mxc,1,0,0,0,1246
1500,gonadotropin-releasing hormone,gnrh,gonadotropin-releasing hormone,gnrh,1,0,0,0,1246


In [28]:
#@title Xem số trường hợp dự đoán sai
# Calculate TP, FP, FN
TP = sum(final_contigency_table['TP'])
FP = sum(final_contigency_table['FP'])
FN = sum(final_contigency_table['FN'])
M = sum(final_contigency_table['M'])
print(f'True positive: {TP}\n False positve: {FP}\n False negative: {FN}\n Modified: {M}')

True positive: 921
 False positve: 68
 False negative: 131
 Modified: 138


In [29]:
#@title Xem các chỉ số đánh giá model
# Calculate precision, recall, f1
precision = TP/(TP+FP)
recall = TP/(TP+FN)
f1 = 1/((1/precision+1/recall)/2)
print(f'Precision: {precision}\n Recall: {recall}\n F1 score: {f1}\n')

Precision: 0.9312436804853387
 Recall: 0.8754752851711026
 F1 score: 0.90249877511024



In [30]:
#@title kiểm tra trường hợp cụ thể
idx = 1001 #@param {type: "number"}
corpus = "ab3p" #@param ["ab3p", "bioadi"]
sentence = ab3p.documents[idx].passages[1].text
BuildContigencyTable(sentence,corpus,idx)

,longform,shortform,truelongform,trueshortform,TP,FP,FN,M,idx
0,radial arteries,ra,radial arteries,ra,1,0,0,0,1001
1,diltiazem,dilt,diltiazem,dilt,1,0,0,0,1001
2,30 microm of mibefradil,mibe,NaN,NaN,0,0,0,0,1001
3,mibefradil,mibe,mibefradil,mibe,1,0,0,0,1001
4,verapamil+ 30 microm of nitroglycerin,vp-ntg,NaN,NaN,0,0,0,0,1001
5,30 microm of nitroglycerin,vp-ntg,NaN,NaN,0,0,0,0,1001
6,nitroglycerin,vp-ntg,NaN,NaN,0,0,0,0,1001
7,NaN,NaN,verapamil + nitroglycerin,vp-ntg,0,0,0,1,1001


# Kiểm tra model với văn bản mới

In [31]:
print("Nhập đoạn văn")
text = input()

Nhập đoạn văn
Here we provide the frst computational method that can regularly predict protein structures with atomic accuracy even in cases in which no similar structure is known. We validated an entirely redesigned version of our neural network-based model, AlphaFold, in the challenging 14th Critical Assessment of protein Structure Prediction (CASP14)15, demonstrating accuracy competitive with experimental structures in a majority of cases and greatly outperforming other methods. Underpinning the latest version of AlphaFold is a novel machine learning approach that incorporates physical and biological knowledge about protein structure, leveraging multi-sequence alignments, into the design of the deep learning algorithm.


Dự đoán token cho tác vụ sentence classification

In [32]:
PredictTokenSentence(text)

['S',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'S',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

Dự đoán các câu chứa cặp từ viết tắt/định nghĩa

In [33]:
PredictSentenceWithABBR(text)

['We validated an entirely redesigned version of our neural network-based model, AlphaFold, in the challenging 14th Critical Assessment of protein Structure Prediction (CASP14)15, demonstrating accuracy competitive with experimental structures in a majority of cases and greatly outperforming other methods.']

Dự doán token cho tác vụ gán nhãn LFSF trong câu

In [34]:
sentenceIdx = 0 # chỉ số câu dự đoán
sentence = PredictSentenceWithABBR(text)[sentenceIdx]
PredictToken(sentence)

[('[CLS]', 'TAG'),
 ('we', 'O'),
 ('validated', 'O'),
 ('an', 'O'),
 ('entirely', 'O'),
 ('redes', 'O'),
 ('##igned', 'O'),
 ('version', 'O'),
 ('of', 'O'),
 ('our', 'O'),
 ('neural', 'O'),
 ('network', 'O'),
 ('-', 'O'),
 ('based', 'O'),
 ('model', 'O'),
 (',', 'O'),
 ('alpha', 'O'),
 ('##fold', 'O'),
 (',', 'O'),
 ('in', 'O'),
 ('the', 'O'),
 ('challenging', 'O'),
 ('14', 'B-LF'),
 ('##th', 'I-LF'),
 ('critical', 'B-LF'),
 ('assessment', 'I-LF'),
 ('of', 'I-LF'),
 ('protein', 'B-LF'),
 ('structure', 'I-LF'),
 ('prediction', 'L-LF'),
 ('(', 'U-PR'),
 ('cas', 'U-SF'),
 ('##p', 'U-SF'),
 ('##14', 'U-SF'),
 (')', 'U-PR'),
 ('15', 'O'),
 (',', 'O'),
 ('demonstrating', 'O'),
 ('accuracy', 'O'),
 ('competitive', 'O'),
 ('with', 'O'),
 ('experimental', 'O'),
 ('structures', 'O'),
 ('in', 'O'),
 ('a', 'O'),
 ('majority', 'O'),
 ('of', 'O'),
 ('cases', 'O'),
 ('and', 'O'),
 ('greatly', 'O'),
 ('outperform', 'O'),
 ('##ing', 'O'),
 ('other', 'O'),
 ('methods', 'O'),
 ('.', 'O'),
 ('[SEP]', 'TAG

Dự đoán cặp LFSF từ câu

In [35]:
predictLFSFFromSentence(sentence)

,longform,shortform
0,14th critical assessment of protein structure ...,casp14
1,critical assessment of protein structure predi...,casp14
2,protein structure prediction,casp14


Dự đoán cặp LFSF từ đoạn văn

In [36]:
predictLFSFFromPassage(text)

,longform,shortform
0,14th critical assessment of protein structure ...,casp14
1,critical assessment of protein structure predi...,casp14
2,protein structure prediction,casp14
